In [1]:
from typing import Callable, get_origin, Union, get_args
import importlib
from typing_extensions import Annotated
import functools
import inspect
from pathlib import Path
import os
import typer
from skimage import io
import numpy as np
from napari import layers, types
import napari
import npe2
from magicgui.type_map._magicgui import MagicFactory
from magicgui.widgets import Label, Container, Button, create_widget
from napari_cli_test.function import threshold_otsu, threshold_mean

from napari_cli_test import threshold_mean, threshold_otsu, make_cli_executable
from napari_cli_test.main import app

In [2]:
supported_inputs = [
    layers.Image,
    layers.Labels,
]

def parse_napari_image(value: Path) -> layers.Image:
    image = io.imread(value)
    return layers.Image(image, name=Path(value).stem)

def parse_napari_labels(value: Path) -> layers.Labels:
    labels = io.imread(value).astype(np.uint8)
    return layers.Labels(labels, name=Path(value).stem)

def is_optional(annotation):
    return (
        get_origin(annotation) is Union
        and type(None) in get_args(annotation)
    )

_WRITER_DISPATCH = {
    layers.Image: io.imsave,
    layers.Labels: io.imsave,
}

_READER_DISPATCH = {
    layers.Image: parse_napari_image,
    layers.Labels: parse_napari_labels,
}

_OUTPUT_FOLDER_PARAM_NAME = 'output_folder'

In [4]:
plugin_manager = npe2.PluginManager()
plugin_manager.discover()

manifest = plugin_manager.get_manifest('napari-skimage')

In [5]:
for idx, cmd in enumerate(manifest.contributions.commands):
    module_path, function_name = cmd.python_name.split(':')

    # Import the module
    module = importlib.import_module(module_path)

    # Get the function
    function = getattr(module, function_name)

    if inspect.isclass(function):
        continue

    if isinstance(function, MagicFactory):
        widget = function()
        function = widget._function

    try:
        app.command()(make_cli_executable(function))
        print(f"{idx}: {function.__name__} registered as CLI command.")
    except Exception as e:
        print(f"Error registering {function.__name__}: {e}")
        continue

0: farid_filter_widget registered as CLI command.
1: prewitt_filter_widget registered as CLI command.
2: laplace_filter_widget registered as CLI command.
3: gaussian_filter_widget registered as CLI command.
4: frangi_filter_widget registered as CLI command.
5: median_filter_widget registered as CLI command.
6: butterworth_filter_widget registered as CLI command.
7: threshold_widget registered as CLI command.
9: binary_morphology_widget registered as CLI command.
10: morphology_widget registered as CLI command.
11: label_widget registered as CLI command.
12: maths_image_pairs_widget registered as CLI command.
13: simple_maths_widget registered as CLI command.
14: maths_crop_widget registered as CLI command.
15: conversion_widget registered as CLI command.
17: rolling_ball_restoration_widget registered as CLI command.
18: denoise_nl_means_restoration_widget registered as CLI command.
19: peak_local_max_widget registered as CLI command.
20: marching_cubes_widget registered as CLI command.

In [6]:
image1 = np.random.random((100, 100))
image2 = np.random.random((100, 100))

In [7]:
viewer = napari.Viewer()
viewer.add_image(image1, name='Image 1')
viewer.add_image(image2, name='Image 2')

<Image layer 'Image 2' at 0x223d2e93d50>